In [ ]:
!rm -rf sample_data

In [ ]:
%%writefile requirements.txt
langchain
langchain-openai
langchain-community

In [ ]:
!pip install -r requirements.txt

In [ ]:
%%writefile app.py

import os
import pickle
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# File to persist chat history
os.makedirs(".storage", exist_ok=True)
HISTORY_FILE = ".storage/.history.pkl"

# Load history from file or create new
if os.path.exists(HISTORY_FILE):
    with open(HISTORY_FILE, "rb") as f:
        history = pickle.load(f)
else:
    history = ChatMessageHistory()

# Initialize OpenAI model
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    openai_api_key="sk-proj-xxx"
)

# Prompt template with message history
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer concisely and accurately."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# Create chat chain
chain = prompt | llm

# Set up chatbot with history
chatbot = RunnableWithMessageHistory(
    chain,
    lambda _: history,
    input_messages_key="input",
    history_messages_key="history"
)

# Get user input and respond
user_input = input()
response = chatbot.invoke({"input": user_input}, config={"configurable": {"session_id": "ignored"}})
print(response.content)

# Save updated history
with open(HISTORY_FILE, "wb") as f:
    pickle.dump(history, f)


In [ ]:
!python app.py

In [ ]:
!pip install hal9

In [ ]:
!HAL9_TOKEN=H9XXX hal9 deploy . --name langchain-demo